Using rogbas great episode scraper notebook https://www.kaggle.com/robga/google-football-episode-scraper we can use LightGBM to predict the top players actions.

I have an a couple agents nearly identical to this notebook that have scores over 940.



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time

In [ ]:
MIN_FINAL_RATING = 1200 # top submission in a match must have reached this score
num_api_calls_today = 0

In [ ]:
all_files = []
for root, dirs, files in os.walk('./', topdown=False):
    all_files.extend(files)
seen_episodes = [int(f.split('.')[0]) for f in all_files 
                      if '.' in f and f.split('.')[0].isdigit() and f.split('.')[1] == 'json']
print('{} games in existing library'.format(len(seen_episodes)))

In [ ]:
NUM_TEAMS = 3
EPISODES = 90 

BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"
# inital team list

r = requests.post(list_url, json = {"teamId":  5696217}) # arbitrary ID, change to leading ID during challenge

rj = r.json()

teams_df = pd.DataFrame(rj['result']['teams'])
teams_df.sort_values('publicLeaderboardRank', inplace = True)
teams_df.head(6)

In [ ]:
def getTeamEpisodes(team_id):
    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
    global teams_df
    teams_df_new = pd.DataFrame(rj['result']['teams'])
    
    if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
        teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
        teams_df.sort_values('publicLeaderboardRank', inplace = True)
    else:
        print('teams dataframe did not match')
    
    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;
    
    for i in range(len(team_episodes)):
        agents = team_episodes['agents'].loc[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = [a['submissionId'] for a in agents if a['submission']['teamId'] == team_id][0]
        team_episodes.loc[i, 'updatedScore'] = [a['updatedScore'] for a in agents if a['submission']['teamId'] == team_id][0]
        
        if len(agent_scores) > 0:
            team_episodes.loc[i, 'avg_score'] = np.mean(agent_scores)

    for sub_id in team_episodes['submissionId'].unique():
        sub_rows = team_episodes[ team_episodes['submissionId'] == sub_id ]
        max_time = max( [r['seconds'] for r in sub_rows['endTime']] )
        final_score = max( [r['updatedScore'] for r_idx, (r_index, r) in enumerate(sub_rows.iterrows())
                                if r['endTime']['seconds'] == max_time] )

        team_episodes.loc[sub_rows.index, 'final_score'] = final_score
        
    team_episodes.sort_values('avg_score', ascending = False, inplace=True)
    return rj, team_episodes
def saveEpisode(epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])

    # save episode info
    with open('{}_info.json'.format(epid), 'w') as f:
        json.dump([r for r in rj['result']['episodes'] if r['id']==epid][0], f)
global num_api_calls_today

pulled_teams = {}
pulled_episodes = []
start_time = datetime.datetime.now()
r = BUFFER;

while num_api_calls_today < EPISODES:
    # pull team
    top_teams = [i for i in teams_df.id if i not in pulled_teams]
    if len(top_teams) > 0:
        team_id = top_teams[0]
    else:
        break;
        
    # get team data
    team_json, team_df = getTeamEpisodes(team_id); r+=1;
    num_api_calls_today+=1
    print('{} games for {}'.format(len(team_df), teams_df.loc[teams_df.id == team_id].iloc[0].teamName))

    
    team_df = team_df[  (MIN_FINAL_RATING is None or (team_df.final_score > MIN_FINAL_RATING))]
    
    print('   {} in score range from {} submissions'.format(len(team_df), len(team_df.submissionId.unique() ) ) )
    
    team_df = team_df[~team_df.id.isin(pulled_episodes + seen_episodes)]        
    print('      {} remain to be downloaded\n'.format(len(team_df)))
        
    # pull games
    target_team_games = int(np.ceil(EPISODES / NUM_TEAMS))
    if target_team_games + len(pulled_episodes) > EPISODES:
        target_team_games = EPISODES - len(pulled_episodes)
     
    pulled_teams[team_id] = 0
    
    i = 0
    while i < len(team_df) and pulled_teams[team_id] < target_team_games:
        epid = team_df.id.iloc[i]
        if not (epid in pulled_episodes or epid in seen_episodes):
            try:
                saveEpisode(epid, team_json); r+=1;
                num_api_calls_today+=1
            except:
                time.sleep(20)
                i+=1;
                continue;
                
            pulled_episodes.append(epid)
            pulled_teams[team_id] += 1
            try:
                size = os.path.getsize('{}.json'.format(epid)) / 1e6
                print(str(num_api_calls_today) + ': Saved Episode #{} @ {:.1f}MB'.format(epid, size))
            except:
                print('  file {}.json did not seem to save'.format(epid))    
            if r > (datetime.datetime.now() - start_time).seconds:
                time.sleep( r - (datetime.datetime.now() - start_time).seconds)
                

        i+=1;
    print(); print()

In [ ]:

import json
import os
from tqdm import tqdm, notebook

In [ ]:
def convert_observation(observation, fixed_positions=False):

    def do_flatten(obj):
        if type(obj) == list:
            return np.array(obj).flatten()
        return obj.flatten()

    final_obs = []
    
    for obs in observation:
        

        o = []
        if fixed_positions:
            for i, name in enumerate(['left_team', 'left_team_direction',
                                    'right_team', 'right_team_direction']):
                o.extend(do_flatten(obs[name]))
            # If there were less than 11vs11 players we backfill missing values
            # with -1.
            if len(o) < (i + 1) * 22:
                o.extend([-1] * ((i + 1) * 22 - len(o)))
        else:
            o.extend(do_flatten(obs['left_team']))
            o.extend(do_flatten(obs['left_team_direction']))
            o.extend(do_flatten(obs['right_team']))
            o.extend(do_flatten(obs['right_team_direction']))

        # If there were less than 11vs11 players we backfill missing values with
        # -1.
        # 88 = 11 (players) * 2 (teams) * 2 (positions & directions) * 2 (x & y)
        if len(o) < 88:
            o.extend([-1] * (88 - len(o)))

        # ball position
        o.extend(obs['ball'])
        # ball direction
        o.extend(obs['ball_direction'])
        # one hot encoding of which team owns the ball
        if obs['ball_owned_team'] == -1:
            o.extend([1, 0, 0])
        if obs['ball_owned_team'] == 0:
            o.extend([0, 1, 0])
        if obs['ball_owned_team'] == 1:
            o.extend([0, 0, 1])

        active = [0] * 11
        if obs['active'] != -1:
            active[obs['active']] = 1
        o.extend(active)

        game_mode = [0] * 7
        game_mode[obs['game_mode']] = 1
        o.extend(game_mode)
        final_obs.append(o)

        return np.array(final_obs, dtype=np.float32).flatten()


In [ ]:
y=[]
x=[]
filenames = [p for p in os.listdir('.') if 'info' not in p and 'json' in p]

for f in notebook.tqdm(filenames):
    filename = "./" + f

    try:
        with open(filename) as json_file:
            data = json.load(json_file)
    except:
        continue
        
    counter=0


    for team in [0,1]:
        final_score = data['steps'][-2][team]['observation']['players_raw'][0]['score'][0]

        goal=False

        for i in range(2,len(data['steps'])-2,):

            action=data['steps'][i][team]['action']
            y.append(action[0])
            
            obs=data['steps'][i][team]['observation']['players_raw'][0]
            x.append(convert_observation([obs]))
                

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score
def evaluate_model(model,X_test,Y_test):
    preds = model.predict(X_test)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))
    

In [ ]:
x[0].shape

In [ ]:
d_train = lgb.Dataset(np.array(x).reshape((-1,115)), label=y)
params = {}
params['objective'] = 'multiclass'
params['num_classes'] = 19


mod = lgb.train(params, d_train, 100,)

mod.save_model('model.txt')    
evaluate_model(mod,x[:1000],y[:1000])

In [ ]:
def tree_agent(obs):
    try:
        obs1=obs['observation']['players_raw'][0]
    except:
        obs1=obs['players_raw'][0]

    obs1=convert_observation([obs1])

#     action=np.random.choice(np.arange(19),p=(mod.predict([obs1])).flatten())
    action=np.argmax(mod.predict([obs1]).flatten())
    return [int(action)]

In [ ]:
from kaggle_environments import make

env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run([tree_agent,tree_agent])
print('Left player: reward = %s, status = %s, info = %s' % (output[-1][0]['reward'],
                                                            output[-1][0]['status'], output[-1][0]['info']))
env.render(mode='human',width=800, height=600)